In [1]:
import pandas as pd
import numpy as np
import nltk
import sklearn as sk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')
import re
from bs4 import BeautifulSoup
import contractions
from sklearn.linear_model import Perceptron
from sklearn.metrics import classification_report, accuracy_score
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import platform 
import torch
import sys

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/geetshingi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/geetshingi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/geetshingi/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
has_gpu = torch.cuda.is_available()
has_mps = getattr(torch,'has_mps',False)
device = "mps" if getattr(torch,'has_mps',False) \
    else "gpu" if torch.cuda.is_available() else "cpu"

print(f"Python Platform: {platform.platform()}")
print(f"PyTorch Version: {torch.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
print("GPU is", "available" if has_gpu else "NOT AVAILABLE")
print("MPS (Apple Metal) is", "AVAILABLE" if has_mps else "NOT AVAILABLE")
print(f"Target device is {device}")

Python Platform: macOS-13.2.1-arm64-arm-64bit
PyTorch Version: 2.0.0.dev20230212

Python 3.9.16 | packaged by conda-forge | (main, Feb  1 2023, 21:38:11) 
[Clang 14.0.6 ]
Pandas 1.5.3
Scikit-Learn 1.2.1
GPU is NOT AVAILABLE
MPS (Apple Metal) is AVAILABLE
Target device is mps


In [4]:
df = pd.read_csv('text_data_nathan.csv')
df.head()

,Channel,Utterance,Label,file
0,A,Okay.,"fo_o_fw_""_by_bc",2228.txt
1,B,"All right, uh,",b,2228.txt
2,B,feelings on what caused the S and L crisis,sd,2228.txt
3,B,I guess I don't have a real technical knowledg...,sd,2228.txt
4,B,I gather that there where large numbers of sit...,sd,2228.txt


In [5]:
df['context_1'] = ''
df['context_2'] = ''
df['context_3'] = ''
df.head()

,Channel,Utterance,Label,file,context_1,context_2,context_3
0,A,Okay.,"fo_o_fw_""_by_bc",2228.txt,,,
1,B,"All right, uh,",b,2228.txt,,,
2,B,feelings on what caused the S and L crisis,sd,2228.txt,,,
3,B,I guess I don't have a real technical knowledg...,sd,2228.txt,,,
4,B,I gather that there where large numbers of sit...,sd,2228.txt,,,


In [6]:
prev_1 =  ''
prev_2 =  ''
prev_3 =  ''
prev_file = None
for index, row in df.iterrows():
    if prev_file != row['file']:
        prev_file = row['file']
        prev_1 = row['Utterance']
        prev_2 = ''
        prev_3 = ''
        continue
    df.loc[index, 'context_1'] = prev_1
    df.loc[index, 'context_2'] = prev_2
    df.loc[index, 'context_3'] = prev_3
    prev_3 = prev_2
    prev_2 = prev_1
    prev_1 = row['Utterance']
df.head()

,Channel,Utterance,Label,file,context_1,context_2,context_3
0,A,Okay.,"fo_o_fw_""_by_bc",2228.txt,,,
1,B,"All right, uh,",b,2228.txt,Okay.,,
2,B,feelings on what caused the S and L crisis,sd,2228.txt,"All right, uh,",Okay.,
3,B,I guess I don't have a real technical knowledg...,sd,2228.txt,feelings on what caused the S and L crisis,"All right, uh,",Okay.
4,B,I gather that there where large numbers of sit...,sd,2228.txt,I guess I don't have a real technical knowledg...,feelings on what caused the S and L crisis,"All right, uh,"


In [8]:
df.to_csv('text_data_context.csv', index = False)

In [ ]:
# from sklearn.model_selection import train_test_split
# from sklearn.utils import shuffle

# shuffled_df = shuffle(df)

# x_train, x_test, y_train, y_test = train_test_split(
#                     shuffled_df['Utterance'], shuffled_df['Label'], 
#                     train_size = 0.95, test_size = 0.05, shuffle=True)

In [9]:
test_df = df[df['file'].isin(['2121.txt', '2131.txt', '2151.txt', '2229.txt', '2335.txt', '2434.txt', '2441.txt', '2461.txt', '2503.txt', '2632.txt', '2724.txt', '2752.txt', '2753.txt', '2836.txt', '2838.txt', '3528.txt', '3756.txt', '3942.txt', '3994.txt'])]
test_df.head()

,Channel,Utterance,Label,file,context_1,context_2,context_3
195662,A,Okay.,"fo_o_fw_""_by_bc",2836.txt,,,
195663,A,"Uh, far as, you know, crime in the cities, I'm...",sv,2836.txt,Okay.,,
195664,A,"Uh, probably the biggest thing we've got going...",sv,2836.txt,"Uh, far as, you know, crime in the cities, I'm...",Okay.,
195665,A,and theft and probably murder are the two top ...,sv,2836.txt,"Uh, probably the biggest thing we've got going...","Uh, far as, you know, crime in the cities, I'm...",Okay.
195666,B,Uh-huh.,b,2836.txt,and theft and probably murder are the two top ...,"Uh, probably the biggest thing we've got going...","Uh, far as, you know, crime in the cities, I'm..."


In [10]:
test_df.shape

(4078, 7)

In [11]:
train_df = df[~df['file'].isin(['2121.txt', '2131.txt', '2151.txt', '2229.txt', '2335.txt', '2434.txt', '2441.txt', '2461.txt', '2503.txt', '2632.txt', '2724.txt', '2752.txt', '2753.txt', '2836.txt', '2838.txt', '3528.txt', '3756.txt', '3942.txt', '3994.txt'])]
train_df.head()

,Channel,Utterance,Label,file,context_1,context_2,context_3
0,A,Okay.,"fo_o_fw_""_by_bc",2228.txt,,,
1,B,"All right, uh,",b,2228.txt,Okay.,,
2,B,feelings on what caused the S and L crisis,sd,2228.txt,"All right, uh,",Okay.,
3,B,I guess I don't have a real technical knowledg...,sd,2228.txt,feelings on what caused the S and L crisis,"All right, uh,",Okay.
4,B,I gather that there where large numbers of sit...,sd,2228.txt,I guess I don't have a real technical knowledg...,feelings on what caused the S and L crisis,"All right, uh,"


In [12]:
train_df.shape

(195662, 7)

In [13]:
len(train_df)

195662

In [14]:
import torch
from transformers import RobertaTokenizer, RobertaModel
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
model = RobertaModel.from_pretrained("roberta-base")
from tqdm import tqdm
def extract_roberta(list_of_sentences=[], batch_size=32):
    cls_hidden_states = []
    if len(list_of_sentences)<batch_size: #go longer ones we should batch
        tknzed= tokenizer(list_of_sentences, return_tensors="pt",padding=True)
        b=model(**tknzed)
        cls_hidden_states = torch.squeeze(b.last_hidden_state[:,0,:])
    else:
        for i in tqdm(range(0,len(list_of_sentences),batch_size)):    
            tknzed= tokenizer(list_of_sentences[i:min(len(list_of_sentences),i+batch_size)], return_tensors="pt",padding=True)
            b=model(**tknzed)
            cls_hidden_states.append(torch.squeeze(b.last_hidden_state[:,0,:]))
    return cls_hidden_states

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [21]:
for i in range(0, len(train_df), 1000):
    print(i)
    reduced_train_df = train_df[i: i + 1000]
    reduced_x_train_roberta_utt = extract_roberta(list(reduced_train_df['Utterance']))
    reduced_x_train_utt_torch = torch.cat(reduced_x_train_roberta_utt)
    reduced_x_train_features = reduced_x_train_utt_torch.cpu().detach().numpy()
    if i != 0:
        x_train_features_utt = np.concatenate((x_train_features_utt, reduced_x_train_features), axis = 0)
    else:
        x_train_features_utt = reduced_x_train_features
    del reduced_x_train_roberta_utt, reduced_x_train_utt_torch, reduced_x_train_features
    
    reduced_x_train_roberta_utt = extract_roberta(list(reduced_train_df['context_1']))
    reduced_x_train_utt_torch = torch.cat(reduced_x_train_roberta_utt)
    reduced_x_train_features = reduced_x_train_utt_torch.cpu().detach().numpy()
    if i != 0:
        x_train_features_context1 = np.concatenate((x_train_features_context1, reduced_x_train_features), axis = 0)
    else:
        x_train_features_context1 = reduced_x_train_features
    del reduced_x_train_roberta_utt, reduced_x_train_utt_torch, reduced_x_train_features
    
    reduced_x_train_roberta_utt = extract_roberta(list(reduced_train_df['context_2']))
    reduced_x_train_utt_torch = torch.cat(reduced_x_train_roberta_utt)
    reduced_x_train_features = reduced_x_train_utt_torch.cpu().detach().numpy()
    if i != 0:
        x_train_features_context2 = np.concatenate((x_train_features_context2, reduced_x_train_features), axis = 0)
    else:
        x_train_features_context2 = reduced_x_train_features
    del reduced_x_train_roberta_utt, reduced_x_train_utt_torch, reduced_x_train_features
    
    reduced_x_train_roberta_utt = extract_roberta(list(reduced_train_df['context_3']))
    reduced_x_train_utt_torch = torch.cat(reduced_x_train_roberta_utt)
    reduced_x_train_features = reduced_x_train_utt_torch.cpu().detach().numpy()
    if i != 0:
        x_train_features_context3 = np.concatenate((x_train_features_context3, reduced_x_train_features), axis = 0)
    else:
        x_train_features_context3 = reduced_x_train_features
    del reduced_x_train_roberta_utt, reduced_x_train_utt_torch, reduced_x_train_features
    
    print(x_train_features_utt.shape)
    print(x_train_features_context1.shape)
    print(x_train_features_context2.shape)
    print(x_train_features_context3.shape)
    del reduced_train_df

0


100%|███████████████████████████████████████████| 32/32 [00:15<00:00,  2.01it/s]


(1000, 768)
(1000, 768)
(1000, 768)
(1000, 768)
1000


100%|███████████████████████████████████████████| 32/32 [00:18<00:00,  1.72it/s]


(2000, 768)
(2000, 768)
(2000, 768)
(2000, 768)
2000


...

(194000, 768)
(194000, 768)
(194000, 768)
(194000, 768)
194000


100%|███████████████████████████████████████████| 32/32 [00:10<00:00,  3.00it/s]


(195000, 768)
(195000, 768)
(195000, 768)
(195000, 768)
195000


100%|███████████████████████████████████████████| 21/21 [00:07<00:00,  2.66it/s]


(195662, 768)
(195662, 768)
(195662, 768)
(195662, 768)


In [23]:
y_train = train_df['Label']
y_train = np.array(y_train)

In [24]:
with open('features/train_features_roberta_context_utt.npy', 'wb') as f:
    np.save(f, x_train_features_utt)

with open('features/train_features_roberta_context_1.npy', 'wb') as f:
    np.save(f, x_train_features_context1)
    
with open('features/train_features_roberta_context_2.npy', 'wb') as f:
    np.save(f, x_train_features_context2)
    
with open('features/train_features_roberta_context_3.npy', 'wb') as f:
    np.save(f, x_train_features_context3)
    
with open('features/train_labels_roberta_context.npy', 'wb') as f:
    np.save(f, y_train)

In [26]:
for i in range(0, len(test_df), 1000):
    print(i)
    reduced_train_df = test_df[i: i + 1000]
    reduced_x_train_roberta_utt = extract_roberta(list(reduced_train_df['Utterance']))
    reduced_x_train_utt_torch = torch.cat(reduced_x_train_roberta_utt)
    reduced_x_train_features = reduced_x_train_utt_torch.cpu().detach().numpy()
    if i != 0:
        x_test_features_utt = np.concatenate((x_test_features_utt, reduced_x_train_features), axis = 0)
    else:
        x_test_features_utt = reduced_x_train_features
    del reduced_x_train_roberta_utt, reduced_x_train_utt_torch, reduced_x_train_features
    
    reduced_x_train_roberta_utt = extract_roberta(list(reduced_train_df['context_1']))
    reduced_x_train_utt_torch = torch.cat(reduced_x_train_roberta_utt)
    reduced_x_train_features = reduced_x_train_utt_torch.cpu().detach().numpy()
    if i != 0:
        x_test_features_context1 = np.concatenate((x_test_features_context1, reduced_x_train_features), axis = 0)
    else:
        x_test_features_context1 = reduced_x_train_features
    del reduced_x_train_roberta_utt, reduced_x_train_utt_torch, reduced_x_train_features
    
    reduced_x_train_roberta_utt = extract_roberta(list(reduced_train_df['context_2']))
    reduced_x_train_utt_torch = torch.cat(reduced_x_train_roberta_utt)
    reduced_x_train_features = reduced_x_train_utt_torch.cpu().detach().numpy()
    if i != 0:
        x_test_features_context2 = np.concatenate((x_test_features_context2, reduced_x_train_features), axis = 0)
    else:
        x_test_features_context2 = reduced_x_train_features
    del reduced_x_train_roberta_utt, reduced_x_train_utt_torch, reduced_x_train_features
    
    reduced_x_train_roberta_utt = extract_roberta(list(reduced_train_df['context_3']))
    reduced_x_train_utt_torch = torch.cat(reduced_x_train_roberta_utt)
    reduced_x_train_features = reduced_x_train_utt_torch.cpu().detach().numpy()
    if i != 0:
        x_test_features_context3 = np.concatenate((x_test_features_context3, reduced_x_train_features), axis = 0)
    else:
        x_test_features_context3 = reduced_x_train_features
    del reduced_x_train_roberta_utt, reduced_x_train_utt_torch, reduced_x_train_features
    
    print(x_test_features_utt.shape)
    print(x_test_features_context1.shape)
    print(x_test_features_context2.shape)
    print(x_test_features_context3.shape)
    del reduced_train_df

0


100%|███████████████████████████████████████████| 32/32 [00:13<00:00,  2.38it/s]


(1000, 768)
(1000, 768)
(1000, 768)
(1000, 768)
1000


100%|███████████████████████████████████████████| 32/32 [00:14<00:00,  2.20it/s]


(2000, 768)
(2000, 768)
(2000, 768)
(2000, 768)
2000


...

(3000, 768)
(3000, 768)
(3000, 768)
(3000, 768)
3000


100%|███████████████████████████████████████████| 32/32 [00:11<00:00,  2.88it/s]


(4000, 768)
(4000, 768)
(4000, 768)
(4000, 768)
4000


100%|█████████████████████████████████████████████| 3/3 [00:00<00:00,  4.12it/s]

(4078, 768)
(4078, 768)
(4078, 768)
(4078, 768)


In [27]:
y_test = test_df['Label']
y_test = np.array(y_test)

In [28]:
with open('features/test_features_roberta_context_utt.npy', 'wb') as f:
    np.save(f, x_test_features_utt)

with open('features/test_features_roberta_context_1.npy', 'wb') as f:
    np.save(f, x_test_features_context1)
    
with open('features/test_features_roberta_context_2.npy', 'wb') as f:
    np.save(f, x_test_features_context2)
    
with open('features/test_features_roberta_context_3.npy', 'wb') as f:
    np.save(f, x_test_features_context3)
    
with open('features/test_labels_roberta_context.npy', 'wb') as f:
    np.save(f, y_test)

In [136]:
x_train_utt = np.load('features/train_features_roberta_context_utt.npy', allow_pickle=True)
x_train_context1 = np.load('features/train_features_roberta_context_1.npy', allow_pickle=True)
x_train_context2 = np.load('features/train_features_roberta_context_2.npy', allow_pickle=True)
x_train_context3 = np.load('features/train_features_roberta_context_3.npy', allow_pickle=True)
y_train = np.load('features/train_labels_roberta_context.npy', allow_pickle=True)

x_test_utt = np.load('features/test_features_roberta_context_utt.npy', allow_pickle=True)
x_test_context1 = np.load('features/test_features_roberta_context_1.npy', allow_pickle = True)
x_test_context2 = np.load('features/test_features_roberta_context_2.npy', allow_pickle = True)
x_test_context3 = np.load('features/test_features_roberta_context_3.npy', allow_pickle = True)
y_test = np.load('features/test_labels_roberta_context.npy', allow_pickle=True)

In [ ]:
# for index, row in train_df.iterrows():
#     if row['Channel'] == 'A':
        

In [137]:
print(x_train_utt.shape)
print(x_train_context1.shape)
print(x_train_context2.shape)
print(x_train_context3.shape)

(195662, 768)
(195662, 768)
(195662, 768)
(195662, 768)


In [138]:
print(x_test_utt.shape)
print(x_test_context1.shape)
print(x_test_context2.shape)
print(x_test_context3.shape)

(4078, 768)
(4078, 768)
(4078, 768)
(4078, 768)


In [139]:
# x_train_utt[0]

In [140]:
x_train_utt = np.reshape(x_train_utt, (195662, 1, 768))
x_train_context1 = np.reshape(x_train_context1, (195662, 1, 768))
x_train_context2 = np.reshape(x_train_context2, (195662, 1, 768))
x_train_context3 = np.reshape(x_train_context3, (195662, 1, 768))

In [141]:
x_train_utt.shape

(195662, 1, 768)

In [142]:
# x_train_utt[0]

In [143]:
x_test_utt = np.reshape(x_test_utt, (4078, 1, 768))
x_test_context1 = np.reshape(x_test_context1, (4078, 1, 768))
x_test_context2 = np.reshape(x_test_context2, (4078, 1, 768))
x_test_context3 = np.reshape(x_test_context3, (4078, 1, 768))

In [169]:
x_train = x_train_context3
x_train = np.concatenate((x_train, x_train_context2, x_train_context1, x_train_utt), axis = 1)

In [170]:
# x_train = x_train_utt
# x_train = np.concatenate((x_train, x_train_context1, x_train_context2, x_train_context3), axis = 1)

In [171]:
x_test = x_test_context3
x_test = np.concatenate((x_test, x_test_context2, x_test_context1, x_test_utt), axis = 1)

In [172]:
# x_test = x_test_utt
# x_test = np.concatenate((x_test, x_test_context1, x_test_context2, x_test_context3), axis = 1)

In [173]:
x_train.shape

(195662, 4, 768)

In [174]:
x_test.shape

(4078, 4, 768)

In [175]:
x_train_utt_torch = torch.from_numpy(x_train_utt)
x_train_context1_torch = torch.from_numpy(x_train_context1)
x_train_context2_torch = torch.from_numpy(x_train_context2)
x_train_context3_torch = torch.from_numpy(x_train_context3)

x_test_utt_torch = torch.from_numpy(x_test_utt)
x_test_context1_torch = torch.from_numpy(x_test_context1)
x_test_context2_torch = torch.from_numpy(x_test_context2)
x_test_context3_torch = torch.from_numpy(x_test_context3)

In [151]:
x_train_torch = torch.from_numpy(x_train)
x_test_torch = torch.from_numpy(x_test)

In [152]:
# y_train_torch = torch.from_numpy(y_train)
# y_test_torch = torch.from_numpy(y_test)

In [153]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(y_train)
y_train = le.transform(y_train)
y_test = le.transform(y_test)

In [154]:
zipper = lambda x,y : list(zip(x,y))

In [155]:
y_train.shape

(195662,)

In [156]:
y_test.shape

(4078,)

In [157]:
train_torch = zipper(x_train, y_train)
test_torch = zipper(x_test, y_test)

train_loader = torch.utils.data.DataLoader(train_torch, batch_size=128)
test_loader = torch.utils.data.DataLoader(test_torch, batch_size=128)

In [158]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module): 
    
    def __init__(self):
        super(Net, self).__init__() 
        hidden_1 = 256
        hidden_2 = 10
        self.lstm = nn.LSTM(input_size=768, hidden_size=hidden_1, 
                            batch_first=True, bidirectional = True) 
        self.fc1 = nn.Linear(hidden_1 * 2, 128)
        self.fc2 = nn.Linear(128, 42)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)

    
    def forward(self, x):
        out, hn = self.lstm(x)
        out = self.fc1(self.relu(out[:, -1]))
        out = self.fc2(self.relu(out))
        return out

In [165]:
lstm_model = Net()
print(lstm_model)

Net(
  (lstm): LSTM(768, 256, batch_first=True, bidirectional=True)
  (fc1): Linear(in_features=512, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=42, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
)


In [166]:
# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()
# specify optimizer (stochastic gradient descent) and learning rate = 0.01
optimizer = torch.optim.SGD(lstm_model.parameters(), lr=0.05)

In [167]:
# number of epochs to train the model
n_epochs = 500

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity

for epoch in range(n_epochs):
    # monitor training loss
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    lstm_model.train() # prep model for training
    for data, target in train_loader:
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = lstm_model(data)
        # calculate the loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()*data.size(0)
        
    lstm_model.eval() # prep model for evaluation
    for data, target in test_loader:
        # forward pass: compute predicted outputs by passing inputs to the model
        output = lstm_model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update running validation loss 
        valid_loss += loss.item()*data.size(0)
        
    # print training/validation statistics 
    # calculate average loss over an epoch
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(test_loader.dataset)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}' \
          .format(epoch+1, train_loss, valid_loss))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}). \
        Saving model ...'.format(valid_loss_min, valid_loss))
        
        torch.save(lstm_model.state_dict(), 'lstm_context_1.pt')
        valid_loss_min = valid_loss

Epoch: 1 	Training Loss: 2.151116 	Validation Loss: 2.221570
Validation loss decreased (inf --> 2.221570).         Saving model ...
Epoch: 2 	Training Loss: 2.118629 	Validation Loss: 2.215169
Validation loss decreased (2.221570 --> 2.215169).         Saving model ...
Epoch: 3 	Training Loss: 2.116705 	Validation Loss: 2.210391
Validation loss decreased (2.215169 --> 2.210391).         Saving model ...
Epoch: 4 	Training Loss: 2.113895 	Validation Loss: 2.202138
Validation loss decreased (2.210391 --> 2.202138).         Saving model ...
Epoch: 5 	Training Loss: 2.104172 	Validation Loss: 2.169659
Validation loss decreased (2.202138 --> 2.169659).         Saving model ...
Epoch: 6 	Training Loss: 2.073237 	Validation Loss: 2.127890
Validation loss decreased (2.169659 --> 2.127890).         Saving model ...
Epoch: 7 	Training Loss: 2.039763 	Validation Loss: 2.068361
Validation loss decreased (2.127890 --> 2.068361).         Saving model ...
Epoch: 8 	Training Loss: 2.011462 	Validation 

Epoch: 81 	Training Loss: 0.871938 	Validation Loss: 0.965714
Validation loss decreased (0.981146 --> 0.965714).         Saving model ...
Epoch: 82 	Training Loss: 0.868642 	Validation Loss: 0.974237
Epoch: 83 	Training Loss: 0.864941 	Validation Loss: 0.959886
Validation loss decreased (0.965714 --> 0.959886).         Saving model ...
Epoch: 84 	Training Loss: 0.862271 	Validation Loss: 0.957591
Validation loss decreased (0.959886 --> 0.957591).         Saving model ...
Epoch: 85 	Training Loss: 0.858921 	Validation Loss: 0.954215
Validation loss decreased (0.957591 --> 0.954215).         Saving model ...
Epoch: 86 	Training Loss: 0.855117 	Validation Loss: 0.929642
Validation loss decreased (0.954215 --> 0.929642).         Saving model ...
Epoch: 87 	Training Loss: 0.853284 	Validation Loss: 0.940000
Epoch: 88 	Training Loss: 0.849589 	Validation Loss: 0.946699
Epoch: 89 	Training Loss: 0.850079 	Validation Loss: 0.927828
Validation loss decreased (0.929642 --> 0.927828).         Sav

Epoch: 190 	Training Loss: 0.689012 	Validation Loss: 0.913038
Epoch: 191 	Training Loss: 0.687787 	Validation Loss: 0.899646
Epoch: 192 	Training Loss: 0.687500 	Validation Loss: 0.874599
Epoch: 193 	Training Loss: 0.685069 	Validation Loss: 0.883100
Epoch: 194 	Training Loss: 0.684751 	Validation Loss: 0.879873
Epoch: 195 	Training Loss: 0.684433 	Validation Loss: 0.864592
Epoch: 196 	Training Loss: 0.682528 	Validation Loss: 0.859690
Epoch: 197 	Training Loss: 0.681715 	Validation Loss: 0.885393
Epoch: 198 	Training Loss: 0.681175 	Validation Loss: 0.839254
Epoch: 199 	Training Loss: 0.680998 	Validation Loss: 0.821023
Validation loss decreased (0.834034 --> 0.821023).         Saving model ...
Epoch: 200 	Training Loss: 0.679154 	Validation Loss: 0.824142
Epoch: 201 	Training Loss: 0.677712 	Validation Loss: 0.871741
Epoch: 202 	Training Loss: 0.678507 	Validation Loss: 0.848819
Epoch: 203 	Training Loss: 0.678442 	Validation Loss: 0.857247
Epoch: 204 	Training Loss: 0.677486 	Valid

Epoch: 316 	Training Loss: 0.596755 	Validation Loss: 0.943737
Epoch: 317 	Training Loss: 0.595831 	Validation Loss: 0.956550
Epoch: 318 	Training Loss: 0.594718 	Validation Loss: 0.889610
Epoch: 319 	Training Loss: 0.593908 	Validation Loss: 0.993703
Epoch: 320 	Training Loss: 0.593936 	Validation Loss: 0.911177
Epoch: 321 	Training Loss: 0.593099 	Validation Loss: 0.890396
Epoch: 322 	Training Loss: 0.591119 	Validation Loss: 0.933341
Epoch: 323 	Training Loss: 0.590767 	Validation Loss: 0.890192
Epoch: 324 	Training Loss: 0.590054 	Validation Loss: 0.886942
Epoch: 325 	Training Loss: 0.588363 	Validation Loss: 0.900965
Epoch: 326 	Training Loss: 0.591344 	Validation Loss: 0.898517
Epoch: 327 	Training Loss: 0.589014 	Validation Loss: 0.876326
Epoch: 328 	Training Loss: 0.590151 	Validation Loss: 0.856222
Epoch: 329 	Training Loss: 0.588030 	Validation Loss: 0.921771
Epoch: 330 	Training Loss: 0.586715 	Validation Loss: 0.910062
Epoch: 331 	Training Loss: 0.588481 	Validation Loss: 0

Epoch: 447 	Training Loss: 0.510063 	Validation Loss: 0.936073
Epoch: 448 	Training Loss: 0.507931 	Validation Loss: 0.946970
Epoch: 449 	Training Loss: 0.509358 	Validation Loss: 0.961207
Epoch: 450 	Training Loss: 0.510268 	Validation Loss: 0.958056
Epoch: 451 	Training Loss: 0.510222 	Validation Loss: 0.978883
Epoch: 452 	Training Loss: 0.505928 	Validation Loss: 0.976006
Epoch: 453 	Training Loss: 0.508408 	Validation Loss: 1.011968
Epoch: 454 	Training Loss: 0.505547 	Validation Loss: 0.980223
Epoch: 455 	Training Loss: 0.502750 	Validation Loss: 0.938200
Epoch: 456 	Training Loss: 0.504952 	Validation Loss: 0.952890
Epoch: 457 	Training Loss: 0.503019 	Validation Loss: 1.031865
Epoch: 458 	Training Loss: 0.504778 	Validation Loss: 0.973251
Epoch: 459 	Training Loss: 0.500442 	Validation Loss: 0.982541
Epoch: 460 	Training Loss: 0.499879 	Validation Loss: 1.037658
Epoch: 461 	Training Loss: 0.499559 	Validation Loss: 0.970357
Epoch: 462 	Training Loss: 0.498318 	Validation Loss: 0

In [168]:
lstm_model = Net()
lstm_model.load_state_dict(torch.load("lstm_context_1.pt"))
lstm_model.eval()

pred_list = torch.zeros(0, dtype=torch.long)
target_list = torch.zeros(0, dtype=torch.long)
train_count = 0
count = 0

train_pred_list = torch.zeros(0, dtype=torch.long)
train_target_list = torch.zeros(0, dtype=torch.long)

for data, target in train_loader:
    output = lstm_model(data)
    _, preds = torch.max(output, 1)
    train_pred_list = torch.cat([train_pred_list, preds.view(-1)])
    train_target_list = torch.cat([train_target_list, target.view(-1)])
#     for x in range(16):
#         if preds[x] == target[x]: 
#             train_count+=1
            
# print("Training Accuracy =", train_count / len(train_loader.dataset))
train_result = classification_report(train_pred_list.numpy(), 
                                     train_target_list.numpy())
print("Training Classification report: \n", train_result)
            
for data, target in test_loader:
    output = lstm_model(data)
    _, preds = torch.max(output, 1) 
    pred_list = torch.cat([pred_list, preds.view(-1)])
    target_list = torch.cat([target_list, target.view(-1)])
#     for x in range(16):
#         if preds[x] == target[x]: 
#             count+=1

# print("Testing Accuracy =", count / len(test_loader.dataset))

result = classification_report(pred_list.numpy(), target_list.numpy())
print("Testing Classification report: \n", result)

Training Classification report: 
               precision    recall  f1-score   support

           0       0.79      0.78      0.78     15158
           1       0.30      0.23      0.26       928
           2       0.38      0.56      0.45        63
           3       0.34      0.81      0.48       231
           4       0.15      0.22      0.18       646
           5       0.36      0.62      0.46      6285
           6       0.00      0.00      0.00         0
           7       0.26      0.49      0.34       377
           8       0.09      0.46      0.16        70
           9       0.00      0.00      0.00         0
          10       0.94      0.80      0.87     44262
          11       0.17      0.35      0.23       322
          12       0.55      0.86      0.67      3003
          13       0.00      0.00      0.00         1
          14       0.03      0.21      0.06       145
          15       0.71      0.79      0.75       928
          16       0.45      0.63      0.53    